### 要約
要約には２種類の方法があります。
- 抽出型要約
- 抽象型要約（生成型要約とも）

歴史的には、抽出型要約から抽象型要約が発展した流れがあります。  
seq2seqから始まって、BERTの登場、そしてT5/BARTへと発展していきます。  
上記、それぞれのモデルで要約に特化したモデルが公開されています。例えばBERTSUMなどです。  
BARTから派生したものとして、PEGASUSなどがあります。  
BARTはTransformerのエンコーダ・デコーダモデルです。

#### 要約のタスク
- 抽出型要約：情報抽出のタスク
- 抽象型要約：テキスト生成のタスク

要約のタスクには、次に挙げるような分類も存在します。
- 複数文書の要約 (Multi-Document Summarization)
- クエリフォーカス要約 (Query Focused Summarization)
- ヘッドライン生成 (Headline Generation)
- キーフレーズ抽出 (Keyphrase Extraction)

実務として要約を扱う場合のほとんどが、上記のような条件付きの要約になります。  

#### 要約の精度を測る
抽象型要約は、テキスト生成のタスクでした。  
前回の講義で扱ったように、テキスト生成を行う場合のデコード方法には複数あります。  
その中で、生成されたテキストの人間らしさのようなことも扱いました（ビームサーチとサンプリング）。  
要約においても同様ですが、さらに情報抽出という観点を加えて、精度を測る必要があります。

#### 精度指標
要約の指標と言えば、まずはこの２つです。
- ROUGE：カバー率の評価
- BLEU：一致率の評価

ROUGEは、人間が作成した要約における単語（や単語の組み合わせ）が、生成された要約でどれだけ出現するかを計算します。  
この指標は、生成された要約が長ければ良いスコアを出します。

BLEUは、ROUGEと逆に、生成された要約における単語（や単語の組み合わせ）が、人間の作成した要約でどれだけ出現するかを計算します。  
この指標は、生成された要約が短ければ良いスコアを出します。

実際には、要約の指標としてはROUGEが使われます。  
要約においては長さの制限があることがほとんどで、良いスコアを出すために長いテキストを生成することがそもそも出来ないためです。  
BLEUは要約で使われない訳ではありませんが、指標としては翻訳タスクのメジャーな指標です。

#### 意味を考慮した精度指標

上記の2つは2000年代前半に提案された指標ですが、2020年に提案された方法が意味を考慮した指標として最近注目されています。  
**「要約から質問文を生成し、入力文書・質問文の対が出す答えと、要約・質問文の対が出す答えの間の一致度を測る」**  
この指標をQAGS（Question Answering and Generation for Summarization）と呼びます。読み方は"kags"です。  


![image.png](https://storage.googleapis.com/zenn-user-upload/ea13d322fbd2-20220531.png)

評価の流れは、次の３ステップから成ります。

1. 要約から質問文を生成
2. 入力文書・質問文、要約文書・質問文の組み合わせで、それぞれの質問応答
3. 回答内容の一致度の評価

ROUGEなどとは全く別物の複雑な方法になっています。  
ROUGEなどよりも人間の評価との相関があるという実験結果が着目されています。  

#### モデルを使って、モデルを評価する
QASGでは、質問文の生成・回答の生成で二つのテキスト生成のモデルが必要になります。  
モデルを評価するのに、その他のモデルを使う指標は、比較的新しい考え方です。  
基本的には人間との評価と相関がある指標が採用されていきますが、それに沿った指標を作ることができています。  
要約に限らず、他のモデルを活用して、特定のモデルの挙動を確かめる傾向があるので、ここで流れを確認しておきましょう。  

今回は、次のモデルを使います。
- 評価対象の要約モデル：tsmatz/mt5_summarize_japanese
- 質問文の生成：sonoisa/t5-base-japanese-question-generation
- 回答の生成：tsmatz/roberta_qa_japanese

現時点では、質問文の生成を行う日本語モデルは、正解を与える必要がありました。  
今回は、要約する文書と合わせて正解の設定を行います。


In [ ]:
!pip install transformers sentencepiece gradio

In [307]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer, AutoModelForSeq2SeqLM, RobertaForQuestionAnswering

# 評価対象の要約モデル
tokenizer_sum = AutoTokenizer.from_pretrained("tsmatz/mt5_summarize_japanese")
model_sum = AutoModelForSeq2SeqLM.from_pretrained("tsmatz/mt5_summarize_japanese")

# 質問文の生成
tokenizer_gen_q = T5Tokenizer.from_pretrained("sonoisa/t5-base-japanese-question-generation")
model_gen_q = T5ForConditionalGeneration.from_pretrained("sonoisa/t5-base-japanese-question-generation")

# 回答の生成
tokenizer_qa = AutoTokenizer.from_pretrained("tsmatz/roberta_qa_japanese")
model_qa = RobertaForQuestionAnswering.from_pretrained("tsmatz/roberta_qa_japanese")

In [345]:
# 要約対象のテキスト１
# https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%B3%E3%83%91%E3%83%B3%E3%83%9E%E3%83%B3
ARTICLE = """
ポケットモンスターの原点は、1996年2月27日に発売されたゲームボーイ用ソフト『ポケットモンスター 赤・緑』である。
開発元はゲームフリーク。コンセプトメーカーにしてディレクターを務めたのは、同社代表取締役でもある田尻智。
この作品が小学生を中心に、口コミから火が点き大ヒットとなり、以降も多くの続編が発売されている（詳しくは「ポケットモンスター（ゲーム）」を参照）。
ゲーム本編作品だけでなく、派生作品や関連作品が数多く発売されている（詳しくはポケットモンスターの関連ゲームを参照）。

ポケモンはゲームのみならず、アニメ化、キャラクター商品化、カードゲーム、アーケードゲームと様々なメディアミックス展開がなされ、日本国外でも人気を獲得している。

ポケモン関連ゲームソフトの累計出荷数は、全世界で2017年11月時点で3億本以上[1]、2022年3月時点で4億4000万本以上に達している[2]。
その中で、メインシリーズの累計販売本数は2016年2月時点での最新作、ニンテンドー3DS『オメガルビー・アルファサファイア』までの25作品で2億100万本となる[3]。
"""

In [346]:
# 要約の実行
inputs = tokenizer_sum("summarize: " + ARTICLE, return_tensors="pt")
outputs = model_sum.generate(
    inputs["input_ids"],
    max_new_tokens=300,  # 生成数の上限
    min_length=150,  # 生成数の下限
    num_beams=5  # ビームサーチの設定
)

In [347]:
# 精度を評価するモデルの要約結果
ARTICLE_SUM = tokenizer_sum.decode(outputs[0], skip_special_tokens=True)
print(ARTICLE_SUM)

ゲーム本編「ポケットモンスター 赤・緑」の累計販売本数が、全世界で3億万本以上に達し、世界各地で人気を獲得している。ゲームボーイ用ソフトは、2017年2月27日に発売されたゲームソフトで2億4000万本の売り上げが、過去最多を記録した。このほか、人気が点き大ヒットとなり、世界的に人気を集めているゲームフリークのディレクターを務めた田尻智英氏が、同社の創業者だった。 ポケモン関連ソフトが「ポケモン」と名付けられた。これにより、作品や関連作品が世界中で話題を集めた、ゲームの本編作品と関連作が次々と発表された。さらに、新作シリーズが話題となっている 。


In [348]:
# 質問を生成するモデルに与える必要がある正解単語の設定
# 要約結果から選ぶ
ANSWER = ["2月27日", "ポケットモンスター 赤・緑"]

In [349]:
# 質問文の生成
answer_context_list = [(answer, ARTICLE_SUM) for answer in ANSWER]  # 解答を質問生成する元となる文（要約結果）とセットにする。

generated_questions = []

for answer, context in answer_context_list:
  # モデルに入力可能な形式に変換する
  # 「answer: 」と「context: 」を使った形式に変換にする
  input = tokenizer_gen_q(f"answer: {answer} context: {context}", return_tensors="pt")

  # 質問文を生成する
  output = model_gen_q.generate(
    input['input_ids'],
    max_new_tokens=100,
    num_beams=4  # ビームサーチの設定
  )

  # 生成された問題文のトークン列を文字列に変換する。
  output = tokenizer_gen_q.decode(output[0], skip_special_tokens=True)

  generated_questions.append(output)

In [350]:
print(generated_questions)

['ゲームボーイ用ソフトの「赤・緑」はいつ発売されましたか?', '全世界で3億本以上売れたゲームはどれですか?']


In [351]:
def extract_answer(question, text):
    """質問応答

    Args
        question: str
            質問文のテキスト
        text: str
            質問に回答するために参照するテキスト
    
    Returns
        answer: str
            回答のテキスト
    """
    inputs = tokenizer_qa(question, text, return_tensors="pt")  # tokenizerには複数のテキストを与える
    
    # 正解箇所の予測
    outputs = model_qa(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # 予測結果の開始と終了のインデックスを取得
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    
    # tokenizerの結果から正解を抽出する
    input_ids = inputs["input_ids"].tolist()[0]
    
    answer = tokenizer_qa.decode(input_ids[answer_start:answer_end])
    # FYI: 上記の処理は、下記の処理と同じ結果になる（他では下を使っていることもある）
    # ids->token->string
    # answer = tokenizer_qa.convert_tokens_to_string(
    #     tokenizer_qa.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    # )

    return answer


for q in generated_questions:
    answer_article = extract_answer(q, ARTICLE)
    answer_sum = extract_answer(q, ARTICLE_SUM)

    print(f"質問: {q}")
    print(f"記事からの回答: {answer_article}")
    print(f"要約からの回答: {answer_sum}")

質問: ゲームボーイ用ソフトの「赤・緑」はいつ発売されましたか?
記事からの回答: 、1996年2月27日に発売された
要約からの回答: 、2017年2月27日に発売された
質問: 全世界で3億本以上売れたゲームはどれですか?
記事からの回答: ポケモン関連ゲームソフト
要約からの回答: 「ポケットモンスター 赤・緑」


#### 人力の評価
上記の結果は評価するにも質問の数が足りませんが、これを人力でやることも多々あります。  
評価者に質問を与え、モデルの要約結果を見ながら答えてもらう方法です。  
QAGSを人手でやるようなもので、丁寧に評価することが可能です。  
この方法では評価者は単純び質問に答えるだけなので、評価方法の習得などは必要がありません。  


一方で、評価を専門的に行う立場の人（テスター）は、次の観点での5段階評価などを行います。
- 可読性 (Readability) 
- 流暢さ (Fluency) 
- 情報量の多さ (Informative) 
- 内容の一貫性 (Consistency)


#### こういうレベルからスタートする
開発の現場では、「ちょっとさ、要約のモデル作ってみてくれない」と軽いノリで案件が生まれたりします。  
そんな時は、「動かしてみました」と軽いノリで返したりします。  
その内容からもう少しやってみようなどの話し合いがあったりします。  

上記の例は個人的な経験ですが、実際にNLPのモデルを使って何かをやってみようというと、既存で使えるモデルを動かしてみることがから始めます。  
そこで**まず行うのがモデルの評価**で、まぁこんなもんかとか、ここ弱いなとかを確認します。  

ここから、ファインチューニングのためのデータの予算がついたり、アノテーター（学習データを作る人）・テスター（モデルを評価する人）のリソースが確保できたりします。  
どこまでを実証実験・PoCと呼ぶかは会社次第ですが、ベンチャーですとデモを作って投資家にプレゼンするというのが日常的な流れでした。
ベンチャーでなくともプロジェクトとして予算がついていれば、同じようなデモを行います。 

ChatGPTなどのインパクトは今年くらいは持つと思います。  
ぜひ、経営者の目線がAIに向いているタイミングで、デモ開発の予算つけてもらって動いてみたりしてください。  
自社でユニークなデータを持っている場合は、大学などとの共同研究も企画しやすいです。  




参考文献：
- 要約のサーベイまとめ記事：https://qiita.com/siida36/items/4c0dbaa07c456a9fadd0
- QAGSの提案論文：https://aclanthology.org/2020.acl-main.450/
- ROUGEの解説記事（日本語）：https://qiita.com/icoxfog417/items/65faecbbe27d3c53d212
- QAGSの解説記事（日本語）：https://zenn.dev/ty_nlp/articles/aaad1aec70d53e

## 演習

#### その１:動作確認
次の要約対象のテキストを使って、質問生成・質問応答の流れを実行してみましょう。
- 下記のアンパンマンについてのテキストを要約してください。
- 要約結果から正解を二つ選択して、質問生成用のモデルに与えるためのANSWERを定義してください。

In [333]:
# 要約対象のテキスト２
# https://www.asahi.com/articles/ASR2F328DR2DULEI001.html?iref=com_rnavi_arank_nr01
ARTICLE = """
アンパンマンの生みの親であるやなせたかしの作品で1968年に「バラの花とジョー」、
「チリンの鈴」の絵本や映画にいち早くアンパンマンが登場しているが、この時はまだ人間の姿。
この童話は一年間連載された。[5]アンパンマン、やなせたかしの作品としての、「アンパンマン」は、
PHP研究所が発行する青年向け雑誌『PHP』の通巻第257号に当たる、『こどものえほん』の1969年10月号[6]（同年10月1日刊行）に掲載された青年向け読物、
やなせたかし（絵と文）「アンパンマン」という形が初出である[7][8][9]。
この時期、やなせが『こどものえほん』のために執筆した読物は連載12本の短編で、「アンパンマン」はその6本目の作品であった。
これら12篇は、株式会社山梨シルクセンター（※3年後、株式会社サンリオへ社名変更）より単行本『十二の真珠』名義で1970年に刊行された。

空腹に喘ぐ人の所へ駆け付けて、自らの大事な持ち物であるパンを差し出して食べるよう勧めるという、のちのアンパンマンに通じる物語の骨組みが、
この作品のおいて早くも整えられている[10][6]。
絵本・漫画・アニメなど、のちに描かれるアンパンマンとの大きな違いと言えば、第一に主人公のアンパンマンが普通の人間のおじさんであり[10][6]、
パンは所有物に過ぎなかったことである。
"""

In [334]:
# 質問を生成するモデルに与える必要がある正解単語の設定
# 要約結果から選ぶ
ANSWER = ["", ""]  # ここを変更してください。このままでも動きます。

In [337]:
# 要約の実行
inputs = tokenizer_sum("summarize: " + ARTICLE, return_tensors="pt")
outputs = model_sum.generate(
    inputs["input_ids"],
    max_new_tokens=300,  # 生成数の上限
    min_length=150,  # 生成数の下限
    num_beams=5  # ビームサーチの設定
)

# 精度を評価するモデルの要約結果
ARTICLE_SUM = tokenizer_sum.decode(outputs[0], skip_special_tokens=True)
print(ARTICLE_SUM)

# 質問文の生成
answer_context_list = [(answer, ARTICLE_SUM) for answer in ANSWER]  # 解答を質問生成する元となる文（要約結果）とセットにする。

generated_questions = []

for answer, context in answer_context_list:
  # モデルに入力可能な形式に変換する
  # 「answer: 」と「context: 」を使った形式に変換にする
  input = tokenizer_gen_q(f"answer: {answer} context: {context}", return_tensors="pt")

  # 質問文を生成する
  output = model_gen_q.generate(
    input['input_ids'],
    max_new_tokens=100,
    num_beams=4  # ビームサーチの設定
  )

  # 生成された問題文のトークン列を文字列に変換する。
  output = tokenizer_gen_q.decode(output[0], skip_special_tokens=True)

  generated_questions.append(output)

# 回答生成
for q in generated_questions:
  answer_article = extract_answer(q, ARTICLE)
  answer_sum = extract_answer(q, ARTICLE_SUM)

  print(f"\n")
  print(f"質問: {q}")
  print(f"記事からの回答: {answer_article}")
  print(f"要約からの回答: {answer_sum}")

アンパンマンの生みの親である、やなせたかしの作品として、1970年に出版された青年向け絵本や小説、漫画、アニメ、映画、ドラマなど、さまざまな作品が掲載された。この絵本は、作者のパンを描いた作者が、その名義で名付けられた。この作品の短編で、読者に衝撃を与えた――。 英医学誌「ネイチャー」に掲載された 作品 が5日、出版され、この著称で明らかになった。出版した 出版本 では、 絵本・漫画・アニメ やアニメなど 、絵本の読み物 と絵本、コミック、児童書、小説の作者、英作家、ジョン・ホプキンス大学の集計 で発表した。


質問: やなせたかしの本にはどんな種類の本がありますか?
記事からの回答: 12本の短編で、「アンパンマン」はその6本目の作品であった。 これら12篇は、株式会社山梨シルクセンター(※3年後、株式会社サンリオへ社名変更)より単行本『十二の真珠』
要約からの回答: 短編


質問: やなせたかしの本にはどんな種類の本がありますか?
記事からの回答: 12本の短編で、「アンパンマン」はその6本目の作品であった。 これら12篇は、株式会社山梨シルクセンター(※3年後、株式会社サンリオへ社名変更)より単行本『十二の真珠』
要約からの回答: 短編


#### その２：デモ実装
質問生成・質問応答の流れを実行できるUIをGradioで実装してみましょう。

##### 具体的な作業内容
- イベント用の関数の実装
    - 要約生成の関数作成
    - 質問生成の関数作成
    - 回答生成の関数作成
- `gr.Blocks()`を使ったUIの実装
    - 要約生成のUI作成
    - 質問生成のUI作成
    - 回答生成のUI作成
- イベントの実装（btn.clickなど）
    - 要約生成の実行ボタン作成
    - 質問生成の実行ボタン作成
    - 回答生成の実行ボタン作成


##### 条件
- 要約対象のテキストを入力できる
- 要約を実行し、要約結果を確認できる
- 要約結果を見て、質問生成のための2つの正解を入力できる
- 質問生成のための正解から、質問文を生成できる
- 質問文を確認できる
- 要約前のテキストと要約後のテキストを条件として与えて、質問文への回答を実行できる
- それぞれの結果を確認できる

##### 補足
- 要約生成の関数とUIはシンプル
    - ユーザーからのテキストを１つ受け取り、要約結果を返す
- 質問生成の関数とUIもシンプル
    - ユーザーからテキストを2つ受け取り、それぞれの結果（質問文）を返す
- 回答生成の関数とUIはすこし注意が必要
    - ユーザーからの入力はない
    - 2つの質問文と2つのテキスト（回答のために参照する要約前と要約後のテキスト）の組み合わせ
    - 回答生成を４回実行することになる。
    - 1つのボタンで実行できるようにするためには、回答生成の関数をまとめて処理する関数が必要

##### 参考UI
![layout](https://s3.gifyu.com/images/teset.png)

#### その３:テスター作業
上記で実装したUIのexamplesを作ってみましょう。

まず次の２点を行なってください。
- 要約対象のテキストを探してきて、実際に要約を試してみてください
- 正解の選定、質問文の選定、回答の生成までの確認を行なってください

ユーザーが入力が必要なのは、要約対象のテキストと２つの正解です。  
これをexamplesとして表示させて、examplesのデータをクリックすると入力されるようにしてください

##### ![eg](https://s3.gifyu.com/images/-2023-02-15-4.24.22.png)



#### ライブラリのインポート・モデルのロードなど

In [ ]:
import gradio as gr
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer, AutoModelForSeq2SeqLM, RobertaForQuestionAnswering

# 0.モデルのロード, Examplesの準備
# TODO 評価対象の要約モデルをロード


# TODO 質問文の生成モデルをロード


# TODO 回答の生成モデルをロード

# Example 1
eg_text_1 = """
ポケットモンスターの原点は、1996年2月27日に発売されたゲームボーイ用ソフト『ポケットモンスター 赤・緑』である。
開発元はゲームフリーク。コンセプトメーカーにしてディレクターを務めたのは、同社代表取締役でもある田尻智。
この作品が小学生を中心に、口コミから火が点き大ヒットとなり、以降も多くの続編が発売されている（詳しくは「ポケットモンスター（ゲーム）」を参照）。
ゲーム本編作品だけでなく、派生作品や関連作品が数多く発売されている（詳しくはポケットモンスターの関連ゲームを参照）。

ポケモンはゲームのみならず、アニメ化、キャラクター商品化、カードゲーム、アーケードゲームと様々なメディアミックス展開がなされ、日本国外でも人気を獲得している。

ポケモン関連ゲームソフトの累計出荷数は、全世界で2017年11月時点で3億本以上[1]、2022年3月時点で4億4000万本以上に達している[2]。
その中で、メインシリーズの累計販売本数は2016年2月時点での最新作、ニンテンドー3DS『オメガルビー・アルファサファイア』までの25作品で2億100万本となる[3]。
"""
eg_ans_1_1 = "2月27日"
eg_ans_1_2 = "ポケットモンスター 赤・緑"

# Example 2
eg_text_2 = """
アンパンマンの生みの親であるやなせたかしの作品で1968年に「バラの花とジョー」、
「チリンの鈴」の絵本や映画にいち早くアンパンマンが登場しているが、この時はまだ人間の姿。
この童話は一年間連載された。[5]アンパンマン、やなせたかしの作品としての、「アンパンマン」は、
PHP研究所が発行する青年向け雑誌『PHP』の通巻第257号に当たる、『こどものえほん』の1969年10月号[6]（同年10月1日刊行）に掲載された青年向け読物、
やなせたかし（絵と文）「アンパンマン」という形が初出である[7][8][9]。
この時期、やなせが『こどものえほん』のために執筆した読物は連載12本の短編で、「アンパンマン」はその6本目の作品であった。
これら12篇は、株式会社山梨シルクセンター（※3年後、株式会社サンリオへ社名変更）より単行本『十二の真珠』名義で1970年に刊行された。

空腹に喘ぐ人の所へ駆け付けて、自らの大事な持ち物であるパンを差し出して食べるよう勧めるという、のちのアンパンマンに通じる物語の骨組みが、
この作品のおいて早くも整えられている[10][6]。
絵本・漫画・アニメなど、のちに描かれるアンパンマンとの大きな違いと言えば、第一に主人公のアンパンマンが普通の人間のおじさんであり[10][6]、
パンは所有物に過ぎなかったことである。
"""
eg_ans_2_1 = "アンパンマン"
eg_ans_2_2 = "やなせたかし"


#### イベント用の関数の実装
- 要約生成の関数作成
- 質問生成の関数作成
- 回答生成の関数作成

In [ ]:
# 1. イベント用の関数
def summy(text):
    """要約

    Args
        text: str
            要約対象のテキスト

    Returns
        summarize_text: str
            要約結果のテキスト

    TODO
        処理の実装
    """
    summarize_text = ...
    
    return summarize_text

def generate_questions(answer_1, answer_2, text):
    """質問生成

    Args
        answers: list[str]
            質問生成のための正解単語のリスト
        text: str
            質問文を生成する際に参照するテキスト
        
    Returns
        generated_questions: list[str]
            生成された質問文のリスト

    TODO
        処理の実装
    """
    generated_questions = ...

    return generated_questions

def extract_answer(question, text):
    """質問応答

    Args
        question: str
            質問文のテキスト
        text: str
            質問に回答するために参照するテキスト
    
    Returns
        answer: str
            回答のテキスト

    TODO
        処理の実装
    """
    answer = ...

    return answer

def extract_answer_all(gen_q_1, gen_q_2, source_text, sum_text):
    """extract_answer()をまとめて実行する
    TODO
        処理の実装
    """
    a_source_1 = ...
    a_sum_1 = ...
    a_source_2 = ...
    a_sum_2 = ...

    return a_source_1, a_sum_1, a_source_2, a_sum_2

#### UIの実装
- `gr.Blocks()`を使ったUIの実装
    - 要約生成のUI作成
    - 質問生成のUI作成
    - 回答生成のUI作成
- イベントの実装（btn.clickなど）
    - 要約生成の実行ボタン作成
    - 質問生成の実行ボタン作成
    - 回答生成の実行ボタン作成

In [ ]:
# 2. UIの定義
with gr.Blocks() as demo:
    gr.Markdown("### 1. 要約生成")
    # TODO 要約のための入出力UIの作成
    btn_summy = gr.Button("要約生成")
    
    gr.Markdown("### 2. 質問生成")
    # TODO 質問文生成のための入力UIの作成

    btn_generate_questions = gr.Button("質問生成")

    gr.Markdown("### 3. 回答生成")
    # TODO 質問文を表示するUIの作成
    btn_extract_answer = gr.Button("回答生成")
    # TODO それぞれの回答を表示するUIの作成
    
    # 2. イベント発火
    btn_summy.click(
        summy,
        inputs=...,  # TODO 定義したUIのコンポーネントを与える
        outputs=...  # TODO 定義したUIのコンポーネントを与える
    )
    btn_generate_questions.click(
        generate_questions,
        inputs=...,  # TODO 定義したUIのコンポーネントを与える
        outputs=...  # TODO 定義したUIのコンポーネントを与える
    )
    btn_extract_answer.click(extract_answer_all,
        inputs=...,  # TODO 定義したUIのコンポーネントを与える
        outputs=...  # TODO 定義したUIのコンポーネントを与える
    )

    # Examplesの定義
    gr.Markdown("## Examples")
    gr.Examples()  # TODO Exampleにデータを与えて、表示させる

#### demoの起動

In [ ]:
demo.launch()